In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import submissions dataset
df = pd.read_csv('posts.csv')

In [3]:
# import keyword index dataset
index_df = pd.read_csv('stock_index.csv')

In [4]:
# Combine two datasets
merged_df = df.merge(index_df, on='id')
merged_df = merged_df.drop(['created_utc_y'], axis=1)
merged_df = merged_df.rename(columns={"created_utc_x": "created_utc"})
merged_df['created_utc'] = pd.to_datetime(merged_df['created_utc'], unit='s')
# merged_df.head()

In [5]:
# Replace NaNs with ''
merged_df['selftext'] = merged_df['selftext'].fillna('')
merged_df['title'] = merged_df['title'].fillna('')

In [6]:
pd.DataFrame(merged_df['selftext'].value_counts()).head(5)

selftext
                                                      235958
[removed]                                             161398
[deleted]                                              85446
**Our discord is now live!!! Come on in and sta...       582
 [🐧](https://i.redd.it/9oohuwgcjdh71.jpg) **Bul...       497

In [7]:
pd.DataFrame(merged_df['title'].value_counts()).head(5)

,title
GME,2947
GME Thread,669
Daily Discussion GME Only,660
Today's most mentioned tickers,657
Gme,638


In [8]:
# Replace [removed] and [deleted] with ''
merged_df['selftext_edit'] = merged_df['selftext'].replace(['[removed]', '[deleted]'], '')
pd.DataFrame(merged_df['selftext_edit'].value_counts()).head(3)

selftext_edit
                                                           482802
**Our discord is now live!!! Come on in and sta...            582
 [🐧](https://i.redd.it/9oohuwgcjdh71.jpg) **Bul...            497

In [9]:
# Combine titles and bodies
merged_df['combined'] = merged_df['title'] + ' ' + merged_df['selftext_edit']
merged_df = merged_df.drop(['selftext_edit'], axis=1)

In [10]:
# Add removed/deleted indicator
merged_df['is_removed'] = False
merged_df.loc[merged_df['selftext'] == '[removed]', 'is_removed'] = True

merged_df['is_deleted'] = False
merged_df.loc[merged_df['selftext'] == '[deleted]', 'is_deleted'] = True

In [11]:
stock_symbols = index_df['stock_symbol'].unique().tolist()

In [12]:
# for symbol in stock_symbols:
for symbol in stock_symbols:
    temp_df = merged_df[merged_df['stock_symbol'] == symbol]
    temp_df.to_csv(f'prepared/{symbol}.csv', index=None)